In [ ]:
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
#Khởi tạo đường dẫn
directory ='C:/Users/LINH/Desktop/KLTN/csvs/'

#Khởi tạo list rỗng
mySeries = []
namesofMySeries = []

#Load dữ liệu vào dataframe (df)
for filename in os.listdir(directory):
    df = pd.read_csv(directory+filename)
    
    #Lọc lấy hai cột dữ liệu là thời gian và giá trị trung bình
    df = df.loc[:,["<DTYYYYMMDD>","<Average>"]]
    
    #Đặt thời gian làm chỉ mục
    df.set_index("<DTYYYYMMDD>", inplace=True)
    #Sắp xếp thời gian theo chỉ mục (sắp xếp theo ngày/tháng/năm)
    df.sort_index(inplace=True)
    
    #Thêm dataframe (df) vào list mySeries
    mySeries.append(df)
    #Thêm tên file vào list nameofMySeries
    namesofMySeries.append(filename[:-4])

#print(mySeries)

#Trả về 5 dòng đầu của dataframe
df.head()

In [ ]:
#print(type(mySeries[0]))

print(len(mySeries))
print(len(namesofMySeries))

In [ ]:
#Khởi tạo dictionary
dic = {}

#Chạy vòng lặp cho từng biến (đặt tên là series) thuộc list
for series in mySeries:
    #Đặt key bằng độ dài từng series
    key = len(series)
    
    #Nếu giá trị key chưa có trong dic
    if key not in dic:
        #Giá trị dic[key] bằng 0
        dic[key] = 0
    #Ngược lại, tăng thêm 1
    dic[key] += 1

#In key và giá trị tương ứng trong dic
#Sắp xếp theo thứ tự sắp xếp giảm dần của dic[key]
for key in sorted(dic, key=dic.get, reverse = True):
    print(key, dic[key])

#### Mục tiêu là lấy các file thuộc mySeries có độ dài thỏa mãn độ dài của maxs (2496 và 2494)

In [ ]:
#Tạo list mới để chứa 2 giá trị key là 2496 và 2494
maxs = []

#Đặt điều kiện
#Với key trong dic (theo số lượng giảm dần), lấy 2 giá trị key đầu tiên
for key in sorted(dic, key=dic.get, reverse = True):
    maxs.append(key)
    if len(maxs) == 2:
        break
print(maxs)

#Tạo hai list mới để chứa dữ liệu và tên
newSeries = []
newNameofMySeries = []

#Với i chạy trong khoảng từ 0 đến độ dài mySeries(1784)
for i in range(0, len(mySeries)):
    #Nếu độ dài của dataframe thứ i thuộc mySeries có trong maxs
    if len(mySeries[i]) in maxs:
        
        #Thêm dataframe thứ i, tên thứ i vào newSeries, newNameofMySeries
        newSeries.append(mySeries[i])
        newNameofMySeries.append(namesofMySeries[i])
        
print(newSeries)
#print(newNameofMySeries)

#### Block code phía dưới có nhiệm vụ tìm kiếm file chứa giá trị ngày không đồng nhất với các file còn lại. Sau đó loại bỏ file khác biệt quá nhiều, nếu không có thì tiến hành bổ sung giá trị data bị thiếu, không đồng nhất

In [ ]:
#Khởi tạo dictionary mới
dic2 = {}

#Chạy vòng lặp cho từng biến (đặt tên là series) thuộc list
for series in newSeries:
    #Với date thuộc giá trị chỉ mục của dataframe (series)
    for date in series.index.values:
        #Nếu date không thuộc dic 2
        if date not in dic2:
            #Giá trị date thuộc dic 2 = 0
            dic2[date] = 0
        #Ngược lại, giá trị date thuộc dic 2 tăng thêm 1
        dic2[date] += 1

#print(len(dic2))       

#Với date thuộc dic 2, sắp xếp theo tăng dần của dic2[key]        
for date in sorted(dic2, key=dic2.get):
    print(date, dic2[date])  

#### Có tổng cộng 576 file, gồm 2 nhóm: 349 file chứa tất cả các gtrị ngày, 277 file chứa ít hơn nhóm 349 file 2 ngày dữ liệu. 
#### Dựa vào kết quả ta thấy có 3 nhóm gtrị ngày: nhóm xuất hiện ở tất cả 576 file, nhóm 4 ngày xuất hiện ở 575 file và cuối cùng là nhóm 2 ngày chỉ xuất hiện ở 351 file.
#### Có thể phỏng đoán 2 file xuất hiện ở 2 ngày 23/01/2018 và 24/01/2018 (tăng 4 gtrị dữ liệu vì (351 - 349)*2) là 2 file gây nên việc mất gtrị dữ liệu ở 4 ngày 2015/06/08, 2015/06/09, 2018/09/05, 2018/09/06 (mất 4 gtrị dữ liệu vì (576-575) *4); do chỉ có 2 nhóm file, mỗi nhóm có một độ dài cố định

In [ ]:
dates = []
for date in dic2:
    if (dic2[date] == 575):
        dates.append(date)
print(dates)

#Cách 1: Thêm not vào code cũ...
for date in dates:
    for i in range(0, len(newSeries)):
        if date not in newSeries[i].index.values:
            print(newNameofMySeries[i])
            continue
            
#Cách 2: Tìm file không chứa giá trị ngày cụ thể:
# for i in range(len(newSeries)):
#     series = newSeries[i]
#     if '2018/09/06' not in series.index.values:
#         print(newNameofMySeries[i])

### Thêm data vào missing values

In [ ]:
#Tạo list mới với tên problems_index
problems_index = []

#Tìm biến có giá trị dài nhất và lưu vào max_len
max_len = maxs[0]
longest_series = None

#Với từng biến (series) thuộc list newSeries
for series in newSeries:
    #Nếu độ dài của biến bằng với max_len
    #Gắn giá trị biến series vào biến longest_series
    if len(series) == max_len:
        longest_series = series

#Với i chạy trong khoảng từ 0 đến độ dài của newSeries
for i in range(len(newSeries)):
    #Nếu độ dài của biến thứ i thuộc newSeries khác max_len
    if len(newSeries[i]) != max_len:
        #Thêm giá trị i vào list problems_index
        problems_index.append(i)
        #Đặt lại chỉ mục các series thứ i theo chỉ mục của biến longest series
        newSeries[i] = newSeries[i].reindex(longest_series.index)

#Với từng giá trị biến a thuộc problems_index
#Lấy biến thứ a thuộc newSeries
#Bổ sung các dữ liệu thiếu bằng phép nội suy
for a in problems_index:
    newSeries[a].interpolate(limit_direction="both",inplace=True)

### Block code để kiểm tra sự đồng nhất của tất cả các file

In [ ]:
# #Initialize 2nd dictionary
# dic2 = {}

# #Với series thuộc newSeries
# for series in newSeries:
#     #Với date thuộc giá trị index của series
#     for date in series.index.values:
#         #Nếu date không thuộc dic 2
#         if date not in dic2:
#             #Giá trị date thuộc dic 2 = 0
#             dic2[date] = 0
#         #Ngược lại, giá trị date thuộc dic 2 tăng thêm 1
#         dic2[date] += 1
# #print(len(dic2))        

# #Với date thuộc dic 2, sắp xếp theo giảm dần của key        
# for date in sorted(dic2, key=dic2.get, reverse = True):
#     print(date, dic2[date])

### Chia dữ liệu thành bộ train và bộ test

In [ ]:
#Khởi tạo list với tên trainSet
trainSet = []

#Cho i chạy trong khoảng từ 0 đến 4
for i in range(5):
    trainSet.append([])
    #Cho j chạy trong khoảng từ 0 đến 575
    for j in range(576):
        trainSet[i].append([])
        
        #Với từng giá trị chỉ mục (years) thuộc biến thứ j của newSeries
        for years in newSeries[j].index.values:
            #Nếu 2011 + i bé hơn hoặc bằng 4 kí tự đầu của chỉ mục
            #Nếu 2015 + i lớn hơn hoặc bằng 4 kí tự đầu của chỉ mục
            if 2011 + i <= int(years[:4]) <= 2015 + i:
                
                #Lấy dòng giá trị của biến thứ j trong list newSeries có giá trị chỉ mục thỏa mãn điều kiện 
                #Thêm biến thứ j của list newSeries sau khi lọc dữ liệu vào biến thứ j thuộc biến thứ i trong trainSet
                trainSet[i][j].append((newSeries[j].loc[years]))

In [ ]:
for i in range(5):
    print(trainSet[i][0][:5])

### Kiểm tra dữ liệu lọc có trùng khớp với dữ liệu cũ không

In [ ]:
print(newSeries[0])

In [ ]:
#Khởi tạo list với tên là testSet
testSet = []

#Với i chạy vòng lặp trong khoảng từ 0 đến 575
for i in range(576):
    #Thêm một list rỗng vào testSet
    testSet.append([])
    
    #Với biến years thuộc chỉ mục của dataframe thứ i thuộc newSeries
    for years in newSeries[i].index.values:
        #Lọc dữ liệu của biến thứ i trong list newSeries theo điều kiện
        #Thêm dữ liệu của biến thứ i của newSeries sau khi lọc vào list thứ i của testSet
        if 2020 == int(years[:4]):
            testSet[i].append((newSeries[i].loc[years]))

In [ ]:
print(testSet[0][:5])

### Chuẩn hóa dữ liệu

In [ ]:
#Với i trong khoảng từ 0 đến 4
for i in range(5):
    #Với j trong khoảng từ 0 đến độ dài của biến thứ i thuộc trainSet
    for j in range(len(trainSet[i])):
        trainSet[i][j] = StandardScaler().fit_transform(trainSet[i][j])
        trainSet[i][j] = trainSet[i][j].reshape(len(trainSet[i][j]))

In [ ]:
#Với i trong khoảng từ 0 đến 4
for i in range(5):
    print("max: "+str(max(trainSet[i][0]))+"\tmin: "+str(min(trainSet[i][0])))

In [ ]:
#Với i trong khoảng từ 0 đến độ dài của testSet
for i in range(len(testSet)):
    testSet[i] = StandardScaler().fit_transform(testSet[i])
    testSet[i] = testSet[i].reshape(len(testSet[i]))

In [ ]:
print("max: "+str(max(testSet[0]))+"\tmin: "+str(min(testSet[0])))

# Kmeans Clustering

### Sử dụng PCA

#### PCA là phương pháp giảm số lượng chiều dữ liệu nhưng không làm mất quá nhiều giá trị của bộ dữ liệu đó

#### Để hiểu hơn về PCA, tham khảo một số bài viết, bài giảng sau:

#### https://royalsocietypublishing.org/doi/10.1098/rsta.2015.0202
#### https://youtu.be/FgakZw6K1QQ

In [ ]:
#Khởi tạo biến pca
pca = PCA(n_components=2)

#Khởi tạo list với tên trainSet_transformed
trainSet_transformed = []
#Đặt biến count bằng 0
count = 0

#Với từng list thành phần trainset thuộc list trainSet
for trainset in trainSet:
    trainSet_transformed.append(pca.fit_transform(trainset))
    
    #In tỉ lệ phương sai còn giữ lại
    print(pca.explained_variance_ratio_.cumsum())
    #In bộ dữ liệu mới
    print(trainSet_transformed[count])
    count +=1

In [ ]:
#Trực quan dữ liệu
for i in range(5):
    plt.figure(figsize=(25,10))
    plt.title("Train "+str(i), fontsize= 20)
    plt.scatter(trainSet_transformed[i][:,0], trainSet_transformed[i][:,1], s=300)
    plt.show()

### Tính khoảng cách giữa các cluster với k lần lượt từ (1:24)

In [ ]:
#Tạo list mới
scores = []

#Với i trong khoảng từ 0 đến 4
for i in range(5):
    #Thêm một list rỗng vào list scores
    scores.append([])
    #Gắn giá trị của list thứ i trong list trainSet_transformed vào biến trainset
    trainset = trainSet_transformed[i]
    
    #Với k trong khoảng từ 1 đến 24
    for k in range(1, 25):
        #Chạy phân cụm với số cụm bằng k
        model = KMeans(n_clusters=k, max_iter=5000)
        model.fit(trainset)
        
        #Tính quán tính cụm và thêm giá trị vào list scores
        scores[i].append(model.inertia_)
        #Xuất giá trị quán tính với k cụm
        print("K", k,":", model.inertia_)

### Sử dụng KneeLocator, xác định số lượng k

In [ ]:
#Tạo list tên kvalue
kvalue = []

#Với i trong khoảng từ 0 đến 4
for i in range(5):
    #Tạo biến knee_locator
    knee_locator = KneeLocator(range(1, len(scores[i])+1), scores[i], curve='convex', direction='decreasing', online=True)
    #Thêm vị trí elbow vào kvalue
    kvalue.append(knee_locator.knee)
        #Vẽ sơ đồ quán tính
    knee_locator.plot_knee(figsize=(25,10))
    plt.xlabel("Train "+str(i), fontsize = 16)

In [ ]:
for i in range(5):
    print("K for Train " + str(i) +':', kvalue[i])

### Chạy thuật toán Kmeans với trainSet

In [ ]:
#Tạo list tên Klabels
Klabels = []

#Với i trong khoảng từ 0 đến 4
for i in range(5):
    #Gọi k bằng giá trị của biến thứ i trong list kvalue
    #Gọi trainset bằng giá trị của biến thứ i trong list trainSet_transformed[i]
    k = kvalue[i]
    trainset = trainSet_transformed[i]
    
    #Thực hiện thuật toán gom cụm
    kmeans = KMeans(n_clusters=k, max_iter=5000, random_state=42)
    #Thêm kết quả chia cụm vào list Klabels
    Klabels.append(kmeans.fit_predict(trainset))

### Visualize the result

In [ ]:
#Với i trong khoảng từ 0 tới 4
for i in range(5):
    plt.figure(figsize=(25,10))
    plt.title("Train "+str(i), fontsize= 20)
    plt.scatter(trainSet_transformed[i][:, 0], trainSet_transformed[i][:, 1], c=Klabels[i], s=300)
    plt.show()

### Plotting

In [ ]:
count = 0

#Với Klabel thuộc Klabels
for i in range(5):
    Klabel = Klabels[i] 
    trainset = trainSet[i]   
        
    fig, axs = plt.subplots(3, 2, figsize=(45,25))
    fig.suptitle('Train:' + str(count), fontsize=20, fontweight='bold')
    row_i=0
    column_j=0
    
    #Với từng giá trị label trong bộ Klabel
    for label in set(Klabel):
        cluster = []

        #Với j trong khoảng 576 (độ dài Klabel)
        for j in range(len(Klabel)):        

            #Nếu giá trị thứ j trong Klabel = label
            if(Klabel[j]==label):
                axs[row_i, column_j].plot(trainset[j],c="gray",alpha=0.4)
                cluster.append(trainset[j])

        if len(cluster) > 0:
            axs[row_i, column_j].plot(np.average(np.vstack(cluster),axis=0),c="red")
        axs[row_i, column_j].set_title("Cluster "+str(row_i*2+column_j))
        column_j+=1
        if column_j%2 == 0:
            row_i+=1
            column_j=0

    plt.show()
    count+=1

### Sự phân bổ của file

In [ ]:
#Tạo biến count bằng 0
count = 0

#Với từng biến (Klabel) trong list Klabels
for Klabel in Klabels:
    
    #Biến number bằng giá trị lớn nhất trong set Klabel + 1 
    number = max(set(Klabel)) + 1
    
    #Với i trong khoảng từ 0 đến độ dài của biến number
    #cluster_amount bằng độ dài của biến có giá trị bằng i trong set Klabel
    cluster_amount = [len(Klabel[Klabel==i]) for i in range(number)]
    
    #Đặt tên biểu đồ với cluster_name
    cluster_name = ["cluster_"+str(i) for i in range(number)]
        
    plt.figure(figsize=(15,5))
    plt.title("Cluster Distribution for TrainSet " + str(count))
    plt.bar(cluster_name, cluster_amount)
    plt.show()
    count += 1

### Sắp xếp tên file theo kết quả chia cụm lần 1

In [ ]:
#Tạo list mới
names = []

#Với i trong khoảng từ 0 đến 4
for i in range(5):
    Klabel = Klabels[i] 

    #Tạo dataframe gồm có tên file, kết quả chia cụm theo từng bộ trainSet
    #Đặt kết quả chia cụm làm chỉ mục
    res = pd.DataFrame(zip(newNameofMySeries,Klabel), columns=["Series", "Cluster"]).set_index("Cluster")
    names.append([])

    #Với biến j thuộc set Klabel
    for j in set(Klabel):
        #Biến thứ i trong list names thêm giá trị có res.index bằng biến j từ res
        names[i].append((res[res.index == j]))

        print(names[i][j])

# Chạy thuật toán gom cụm lần 2 với từng cụm ở lần đầu

### Tạo một dataframe mới với kết quả gom cụm

In [ ]:
#Tạo list tên là trainCluster
trainCluster = []

#Với i trong khoảng từ 0 đến 4
for i in range(5):
    Klabel = Klabels[i] 
    trainset = trainSet[i]
    
    #Đưa trainset về dạng dataframe
    datafr = pd.DataFrame(trainset)
    #Đặt kết quả chia cụm làm chỉ mục
    datafr.set_index([Klabel], inplace=True)
    
    #Thêm list rỗng vào list trainCluster
    trainCluster.append([])
    #Biến (list) thứ i trong trainCluster thêm vào giá trị có datafr.index bằng j từ biến datafr
    for j in set(Klabel):
        trainCluster[i].append(np.array(datafr[datafr.index == j]))

### Kiểm tra code

In [ ]:
check = pd.DataFrame(trainSet[1])
check.set_index([Klabels[1]], inplace=True)

check.head(576)

### Chạy PCA

In [ ]:
pca = PCA(n_components=2)
trainCluster_transformed = []

for i in range (5):
    trainCluster_transformed.append([])
    for j in set(Klabels[i]):
        trainCluster[i][j] = np.array(trainCluster[i][j])
        trainCluster_transformed[i].append(pca.fit_transform(trainCluster[i][j]))
        print(trainCluster_transformed[i][j])

In [ ]:
newscores = []

for i in range(5):
    newscores.append([])
    for j in set(Klabels[i]):
        newscores[i].append([])
        traincluster = trainCluster_transformed[i][j]
    
        for k in range(1, 10):
            model = KMeans(n_clusters=k, max_iter=5000)
            model.fit(traincluster)
            newscores[i][j].append(model.inertia_)
            print("K", k,":", model.inertia_)

In [ ]:
newvalue = []
for i in range(5):
    newvalue.append([])
    for j in set(Klabels[i]):
        knee_locator = KneeLocator(range(1, len(newscores[i][j])+1), newscores[i][j], curve='convex', direction='decreasing', online=True)
        #knee_locator.plot_knee(figsize=(25,10))
        newvalue[i].append(knee_locator.knee)
        print("K for Train" + str(i) + " Cluster " + str(j), knee_locator.knee)

In [ ]:
print(newvalue)

In [ ]:
newKlabels = []

for i in range(5):
    newKlabels.append([])
    
    for j in set(Klabels[i]):
        k = newvalue[i][j]
        traincluster = trainCluster_transformed[i][j]
        
        kmeans = KMeans(n_clusters=k, max_iter=5000, random_state=42)
        newKlabels[i].append(kmeans.fit_predict(traincluster))

In [ ]:
for i in range(5):
    for j in set(Klabels[i]):
        plt.figure(figsize=(25,10))
        plt.title('Train ' + str(i) + ' Cluster ' + str(j), fontsize = 20)
        plt.scatter(trainCluster_transformed[i][j][:, 0], trainCluster_transformed[i][j][:, 1], c=newKlabels[i][j], s=300)
        plt.show()

In [ ]:
#Với i trong khoảng từ 0 đến 4
for i in range(5):
    #Với biến j thuộc set(Klabels[i])
    for j in set(Klabels[i]):
        #Gán giá trị
        Klabel = newKlabels[i][j] 
        traincluster = trainCluster[i][j]   

        fig, axs = plt.subplots((max(set(Klabel))+1), figsize=(35, 30))
        fig.suptitle('Train ' + str(i) + ' Cluster ' + str(j), fontsize=20, fontweight='bold')
        #row_i=0
        column_j=0

        #Với từng giá trị label trong bộ Klabel
        for label in set(Klabel):
            cluster = []

            #Với z trong khoảng 0 đến độ dài Klabel (576)
            for z in range(len(Klabel)):        

                #Nếu giá trị thứ j trong Klabel = label
                if(Klabel[z]==label):
                    axs[column_j].plot(traincluster[z],c="gray",alpha=0.4)
                    cluster.append(traincluster[z])

            if len(cluster) > 0:
                axs[column_j].plot(np.average(np.vstack(cluster),axis=0),c="red")
            axs[column_j].set_title("Cluster "+str(column_j), fontsize = 16)
            column_j+=1

        plt.show()

In [ ]:
for i in range(5):
    count = 0
    for Klabel in newKlabels[i]:
        number = max(set(Klabel))+1
        cluster_amount = [len(Klabel[Klabel==j]) for j in range(number)]
        cluster_name = ["cluster_"+str(j) for j in range(number)]
        
        plt.figure(figsize=(15,5))
        plt.title("Cluster Distribution Train " + str(i) + " Cluster " + str(count))
        plt.bar(cluster_name, cluster_amount)
        plt.show()
        count +=1

### Sắp xếp tên file theo kết quả chia cụm lần 2

In [ ]:
newnames = []

for i in range(5):
    newnames.append([])
    for j in set(Klabels[i]):
        name = names[i][j]
        Klabel = newKlabels[i][j]

        #Gắn kết quả chia cụm theo từng bộ trainSet
        name.set_index([Klabel], inplace=True)
        newnames[i].append([])

        #Với z trong khoảng (0,3)
        for z in set(Klabel):
            #List thứ i trong trainCluster append giá trị có res.index == j từ res
            newnames[i][j].append(name[name.index == z])
            
            print(newnames[i][j][z])

### Phân chia dữ liệu theo kết quả chia cụm lần 2

In [ ]:
SmallCluster = []

for i in range(5):
    SmallCluster.append([])
    
    for j in set(Klabels[i]):
        Klabel = newKlabels[i][j]
        traincluster = trainCluster[i][j]
    
        #Đưa trainset về dạng dataframe
        datafr = pd.DataFrame(traincluster)
        #Đặt kết quả chia cụm làm index
        datafr.set_index([Klabel], inplace=True)

        SmallCluster[i].append([])
        #List thứ i trong trainCluster thêm vào giá trị có datafr.index == j từ datafr
        for z in set(Klabel):
            SmallCluster[i][j].append(np.array(datafr[datafr.index == z]))

### Kiểm tra kết quả của đoạn code phân chia dữ liệu lần 2

In [ ]:
check = pd.DataFrame(trainCluster[0][0])
check.set_index([newKlabels[0][0]], inplace=True)

check.head(576)

In [ ]:
print((SmallCluster[0][0]))

### PCA với tỉ lệ phương sai bằng 0.99

In [ ]:
#Tạo biến pca với tỉ lệ phương sai bằng 0.99
pca = PCA(n_components=0.99)
PCACluster = []

#Thực hiện việc giảm chiều dữ liệu theo điều kiện của biến pca
for i in range(5):
    PCACluster.append([])
    
    for j in set(Klabels[i]):
        Klabel = newKlabels[i][j]
        PCACluster[i].append([])
        
        for z in set(Klabel):
            PCACluster[i][j].append(pca.fit_transform(SmallCluster[i][j][z]))
            
            #In ra số lượng dữ liệu mới (dòng dữ liệu) đối với từng cụm nhỏ
            print("Train "+str(i)+" Cluster "+str(j)+" Small Cluster "+str(z)+ ":", len(PCACluster[i][j][z]))

### Dựa vào biểu đồ chuỗi thời gian của các cụm con, chọn lấy các cụm con có ý nghĩa (có hình dáng giống nhau) để tiếp tục phân cụm

In [ ]:
list = [[[1],[0,2],[2],[1]],[[0],[0,2],[2],[2],[1]],[[2],[1],[0],[0],[3]],[[2],[0],[0],[1]],[[1],[0],[2],[0]]]

In [ ]:
#Kiểm tra kết quả trả về theo list
count = 0
for i in range(5):
    for j in set(Klabels[i]):
        for z in list[i][j]:
            print(newnames[i][j][z])
            count+=1

### Thực hiện phân cụm phân cấp (Hierarchical clustering) và thể hiện kết quả thông qua sơ đồ cây

In [ ]:
#Với i trong khoảng từ 0 đến 4
for i in range(5):
    #Với biến j thuộc set(Klabels[i])
    for j in set(Klabels[i]):
        #Với biến z thuộc biến thứ j thuộc biến thứ i của list
        for z in list[i][j]:
            
            traindata = PCACluster[i][j][z]

            plt.figure(figsize=(16, 6))
            plt.xlabel("Train "+str(i)+" Cluster "+str(j)+" Small Cluster "+str(z), fontsize=14)
            plt.ylabel("Euclidean distance", fontsize=14)
            
            merg = linkage(traindata, method = "ward", metric = 'euclidean')
            dendrogram(merg, leaf_rotation= 0, leaf_font_size = 9, distance_sort=True)
            
            plt.show()

### Đối với từng kết quả phân cụm, lấy cặp có khoảng cách Euclidean thấp nhất (gần nhau nhất)

In [ ]:
newlist = [[43,48],[40,58],[8,28],[18,20],[3,5],[24,31],[15,16],[15,18],[25,31],[3,4],[1,3],[16,19],[10,18],[0,1],[19,20],[14,18],[44,56],[19,20],[13,17],[29,49],[29,52],[0,10],[8,28],[5,30]]

### Quan sát các cặp chuỗi dữ liệu

In [ ]:
count = 0
for i in range(5):
    for j in set(Klabels[i]):
        for z in list[i][j]:
            fig, axs = plt.subplots(2, figsize=(14,4))
            fig.suptitle("Train "+str(i)+" Cluster "+str(j)+" Small Cluster "+str(z), fontsize=14)
            fig.tight_layout()
            fig.subplots_adjust(hspace=0.5)

            for a in range(2):
                t = newlist[count][a]
                name = pd.Series.to_string((newnames[i][j][z]).iloc[t], index = False)
                
                axs[a].plot(SmallCluster[i][j][z][t], color = 'red', linewidth=1)
                axs[a].set_title(name)

            plt.show()
            
            count +=1

### Tạo list mới chứa tên và dữ liệu của hai cặp mã chứng khoán đã chọn phía trên (cặp có khoảng cách thấp nhất)

In [ ]:
#Tạo hai list mới
pearsoncc = []
pairname = []
count=0

#Chạy vòng lặp
for i in range(5):
    for j in set(Klabels[i]):
        for z in list[i][j]:
            pearsoncc.append([])
            pairname.append([])
            for a in range(2):
                dict = {}
                t = newlist[count][a]
                
                #Gán giá trị
                name = pd.Series.to_string((newnames[i][j][z]).iloc[t], index = False)
                
                value = (SmallCluster[i][j][z][t])
                dict[name] = value
                datafr = pd.DataFrame(dict)
                pearsoncc[count].append(datafr)
                pairname[count].append(name)
            count +=1

### Đánh giá độ tương quan của mã chứng khoán trong các cặp

In [ ]:
#Tạo list có tên matrix
matrix = []
for i in range(len(pearsoncc)):
    matrix.append([])
    
    #Đánh giá độ tương quan theo phương pháp pearson
    pearsonccvalue = pd.concat(pearsoncc[i], axis = 1)
    res = pearsonccvalue.corr(method='pearson')
    
    #Thêm kết quả vào list
    matrix[i].append(res)
    print(matrix[i])

### Đánh giá độ tương quan giữa mã chứng khoán trong cặp với bộ dữ liệu test

In [ ]:
#Tạo list mới để lưu giá trị chỉ mục
indexes = []
for i in range(len(pairname)):
    indexes.append([])
    name = pairname[i]
    
    for a in range(2):
        #Trả về giá trị chỉ mục của biến có tên giống với biến a thuộc list name
        #Thêm giá trị chỉ mục vào list indexes
        indexes[i].append(newNameofMySeries.index(name[a]))

In [ ]:
#path = 'C:/Users/LINH/Desktop/DS_Essay/Result/'
for i in range(24):
    fig, axs = plt.subplots(2, figsize=(14,4))
    fig.suptitle("TestPair " + str(i), fontsize=14)
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.5)

    for a in range(2):
        t = indexes[i][a]
        name = newNameofMySeries[t]

        axs[a].plot(testSet[t], color = 'red', linewidth=1)
        axs[a].set_title(name)

#     fig.savefig(path + "TestPair" + str(i))
#     plt.close()
        
    plt.show()

In [ ]:
newpearsoncc = []

for i in range(24):
    newpearsoncc.append([])
    for a in range(2):
        dict = {}
        t = indexes[i][a]
                
        value = testSet[t]
        name = newNameofMySeries[t]
                
        dict[name] = value
        datafr = pd.DataFrame(dict)
        newpearsoncc[i].append(datafr)

In [ ]:
matrix = []
for i in range(len(newpearsoncc)):
    matrix.append([])

    newvalue = pd.concat(newpearsoncc[i], axis = 1)
    res = newvalue.corr(method='pearson')
    matrix[i].append(res)
    print(matrix[i])